In [ ]:
import os
import numpy as np
from os.path import expanduser
from fcs_signal_prediction.predict import predict_signal
from fcs_signal_prediction.utils import data_utils as du


%load_ext autoreload
%reload_ext autoreload
%autoreload 2


In [ ]:
DATA_CONVERGE_PROJECT="sd2e-project-43"

data_converge_base = os.path.join(expanduser("~"), 'sd2e-projects', DATA_CONVERGE_PROJECT)
experiment_dir = os.path.realpath(os.path.join(data_converge_base, 'reactor_outputs', 'complete'))
experiment_dir_contents = [os.path.realpath(os.path.join(experiment_dir, x)) for x in os.listdir(experiment_dir)]

experiment_dir_contents

In [ ]:
#experiment_id = "YeastSTATES-CRISPR-Short-Duration-Time-Series-20191208"
experiment_id="YeastSTATES-OR-Gate-CRISPR-Dose-Response"
process_dir = os.path.join(experiment_dir, experiment_id)
experiment_id_dir_contents = [os.path.realpath(os.path.join(process_dir, x)) for x in os.listdir(process_dir)]
experiment_id_dir_contents.sort()
last_process = experiment_id_dir_contents[-1]

In [ ]:
id_col = 'sample_id'
df = du.get_data(last_process, du.get_record(last_process))
meta = du.get_meta(last_process, du.get_record(last_process))
#df = du.get_meta(last_process, du.get_record(last_process))
df = df.merge(meta[[id_col, 'strain_name']])

In [ ]:
channels = list(df.columns)
channels.remove(id_col)
channels.remove("strain_name")

high_control = 'CRISPR_CEN.PK2_positive_control_NOR_00_24864'
low_control = 'CRISPR_CEN.PK2_negative_control_23970'
strain_col = "strain_name"

In [ ]:
predictions = predict_signal(df, experiment_id, low_control, high_control, id_col, channels, strain_col)
mean_prediction = predictions.groupby([id_col]).agg({"predicted_output" : [np.mean, np.std]}).reset_index()
mean_prediction.columns  = mean_prediction.columns.map('_'.join)
mean_prediction = mean_prediction.rename(columns={id_col+"_": id_col})
result = meta.merge(mean_prediction, on=id_col)
result

In [ ]:
def get_log_gfp(df):
    df['log BL1-A'] = df['BL1-A'].apply(lambda x: x+1.0).apply(np.log10)
    return df.replace([np.inf, -np.inf], np.nan).dropna()


log_df = get_log_gfp(df).sample(n=int(10e5))

In [ ]:
import matplotlib.pyplot as plt

samples = log_df.sample_id.unique()
num_samples = len(samples)

fig, ax = plt.subplots(num_samples, 1, figsize=(3, 3*num_samples))

num=100000

high_df = log_df.loc[log_df.strain_name == high_control]#.sample(n=num)
low_df = log_df.loc[log_df.strain_name == low_control]#.sample(n=num)

for i, sample in enumerate(samples):
    #print(i)
    sample_df = log_df.loc[log_df.sample_id == sample]
    
    #sample_df = sample_df.sample(n=min(num, len(sample_df)))

    ax[i].hist(high_df['log BL1-A'], label="high")
    ax[i].hist(low_df['log BL1-A'], label="low")
    ax[i].hist(sample_df['log BL1-A'], label="sample")
    ax[i].set_xlim(0, 5)
    ax[i].text(6, 0, "\n".join(result[result.sample_id==sample][["predicted_output_mean", "predicted_output_std"]].transpose()[0:].to_string().split("\n")[1:]))
    ax[i].set_title(sample)

#    if i >= 9: 
#        break
#fig.legend()

#result[result.sample_id==sample][["predicted_output_mean", "predicted_output_std"]]

